In [383]:
# Importing all the packages
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import argparse

In [384]:
# Define arg parser
seed=200
paser = argparse.ArgumentParser() 
args = paser.parse_args("")
np.random.seed(200)

### Training Part:

- In this part we are going to train the weak Classifier of The adaboost to the given training Data. In the next code block I am going to write a function which taken inputs from the given text file and returns the data frame containg X and Y parameters.

In [385]:
# Enter the data file to train
def file_input():
    return 'multilabel_train_data.txt'
    #return input("Enter File Name")

In [386]:
# Read the data from the file
train_file=file_input()
data=[]
with open(f'{train_file}', 'r') as f:
    data=f.readlines()
data

['6,7,11,12 1:-0.011858 2:-0.061636 3:0.054392 4:0.021137 5:0.089374 6:0.101825 7:-0.044525 8:0.222330 9:0.015575 10:-0.053591 11:0.008859 12:0.244721 13:-0.099625 14:-0.019112 15:0.090234 16:0.202635 17:0.040064 18:0.095590 19:0.170196 20:-0.057035 21:0.016229 22:0.056695 23:0.044291 24:-0.139432 25:-0.072632 26:-0.111325 27:-0.115570 28:-0.221698 29:-0.042896 30:-0.132356 31:-0.148352 32:-0.098631 33:-0.014461 34:0.069131 35:0.010838 36:-0.175787 37:-0.201014 38:-0.016873 39:-0.064189 40:0.148559 41:-0.305013 42:0.008835 43:-0.155010 44:0.020472 45:-0.141353 46:-0.083728 47:-0.284418 48:0.118055 49:0.112069 50:0.137376 51:0.098575 52:0.106653 53:0.094011 54:0.003915 55:0.088345 56:-0.078315 57:0.059667 58:0.135742 59:-0.021467 60:0.005305 61:0.001840 62:-0.058740 63:-0.040525 64:0.034125 65:0.000202 66:-0.110622 67:0.047559 68:0.011386 69:0.029297 70:-0.145037 71:-0.112128 72:-0.027229 73:-0.042747 74:0.109892 75:-0.064474 76:-0.053787 77:-0.113682 78:0.130953 79:0.116326 80:0.038755

In [387]:
# Initialize empty arrays for features (X) and labels (y)
X = []
y = []

In [388]:
# Find the total number of features present and store the labels
feature_tot=0
for line in data:
    parts=line.strip().split()
    #print(parts)
    labels=[]
    for part in parts:
        if ',' in part or ':' not in part:
            if ',' in part:
                labels=part.strip().split(',')
                labels=[int(i) for i in labels]
                y.append(labels)
            else:
                labels=[int(part)]
                y.append(labels)
        if ':' in part:
            temp=part.strip().split(':')
            feature_tot=max(feature_tot,int(temp[0]))
    #print(y)

print("Total number of features present :",feature_tot)
print("Length of  y :",len(y))
print("\nLabels :",y)

Total number of features present : 103
Length of  y : 1500

Labels : [[6, 7, 11, 12], [3, 4, 11, 12], [4, 5, 6, 7, 10, 11, 12], [1, 2, 9, 10], [1, 2, 11, 12], [0, 1], [0, 1, 11, 12], [2, 3, 4, 13], [2, 3, 11, 12], [3, 4, 11, 12], [9, 10, 11, 12], [7, 8], [2, 3, 11, 12, 13], [4, 5], [3, 4, 11, 12], [3, 4, 11, 12], [2, 3, 6, 7, 11, 12], [0, 1, 11, 12], [4, 5, 11, 12], [2, 3, 11, 12], [0, 1, 4, 5], [0, 1], [4, 5, 6, 7, 11, 12], [6, 7, 8, 9], [4, 5, 7, 8, 11, 12], [0, 1, 11, 12], [0, 1], [2, 3, 11, 12], [2, 3, 11, 12], [1, 2, 4, 5, 11, 12], [0, 1, 2, 3, 4, 11, 12], [2, 3, 11, 12, 13], [7, 8], [1, 2, 11, 12], [4, 5], [0, 1, 7, 8], [6, 7, 11, 12], [1, 2, 7, 8, 11, 12], [2, 3, 11, 12], [1, 2, 3, 11, 12], [0, 11, 12], [2, 3, 11, 12], [1, 2], [4, 5, 6, 7, 11, 12], [2, 3, 11, 12], [0, 11, 12], [1, 2, 3], [0, 1, 2, 11, 12], [1, 2, 3, 11, 12], [2, 3, 4, 11, 12, 13], [5, 6, 7], [5, 6, 7, 11, 12], [4, 5, 11, 12], [3, 4, 11, 12], [4, 5], [1, 2, 6, 7, 11, 12], [4, 5, 11, 12], [5, 6, 7, 10, 11, 12], [3

- Encode the labels into one hot variables. This we we can represent multi-class labels. The labels belonging to the datapoint are 1 rest are 0.

In [389]:
# Finding Max number of classes
max_class=0
for i in y:
    max_class=max(max_class,max(i))

print('Total number of classes :',max_class)

Total number of classes : 13


In [390]:
# Encode the variables
temp=[]
for i in y:
    A=np.zeros(max_class)
    for j in i:
        A[j-1]=1
    temp.append(A)
temp=np.array(temp)
y=temp
print("Encoded labels :",y)

Encoded labels : [[0. 0. 0. ... 1. 1. 0.]
 [0. 0. 1. ... 1. 1. 0.]
 [0. 0. 0. ... 1. 1. 0.]
 ...
 [0. 0. 1. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 1. 0.]]


- Extract the feature values for all the datapoints.

In [391]:
# Now creating the X feature column

# Parse each line of the file and store in X and y arrays
for line in data:
    # Split the line by space characters
    parts = line.strip().split()
    
    # The remaining parts are the features
    features = np.zeros(feature_tot)
    for part in parts:
        if ':' in part:
          # Split each feature by ':' character
          id, val = part.split(':')
          # Store the feature value in the dictionary
          features[int(id)-1] = float(val)
    
    # Append the feature dictionary to X array
    X.append(features)

X=np.array(X)
#y=np.array(y)
print(X.shape,y.shape)
print(X)

(1500, 103) (1500, 13)
[[-0.011858 -0.061636  0.054392 ... -0.000844  0.004983  0.115765]
 [-0.018183 -0.045645  0.002001 ...  0.079903  0.065097 -0.165039]
 [ 0.074145 -0.044487  0.048191 ... -0.08563   0.174316 -0.02257 ]
 ...
 [ 0.074028  0.014501 -0.041893 ... -0.072424 -0.039109  0.019538]
 [-0.058193  0.502205 -0.081411 ... -0.017584  0.498207 -0.072177]
 [-0.128468 -0.003593  0.039187 ... -0.062728 -0.048271 -0.124432]]


- From the above codes we have obtained the feautures and the labels. Now I will be applying adaboost algorithm to predict the labels.

- For the weak Classifier I will be testing various Base weak learners such as Decision trees, SVC and other methods and choosing the best weak learner out of them for the adaboost model.

### To be implemented: Separate all the labels. Repeat each sample with each individual label. Maintain the encoding but the number of samples will be equal to the number of labels. As the sample weight is a single scalar quantity so to have different sample weights of same X_train for different class we separate them.

In [392]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
import numpy as np

args.epoch_num=int(input("Enter the number of Epochs"))
 
def calculate_mult(y_true,y_pred):
      temp=[]
      for i in range(y_true.shape[0]):
         #print(y_true[0].shape[0])
         c=np.sum(y_true[i]==y_pred[i])/y_true[i].shape[0]
         '''
         if(c>0.7):
            temp.append(1)
         else:
            temp.append(-1)
          '''
         temp.append(c-1)
      return(np.array(temp))

class AdaboostMH:
  def __init__(self, args):
    self.epoch_num = args.epoch_num
    self.class_num = max_class
    self.alpha = np.zeros(self.epoch_num)
    self.clfs = []
    self.result = []
    self.args = args

  def accuracy(self,y_true,y_pred):
    acc=0
    for i in range(y_true.shape[0]):
       acc += np.sum(y_true[i]==y_pred[i]) / y_pred[i].shape[0]
    return acc/y_true.shape[0]
  
  def learner():
      # Define weak learner
      clf = DecisionTreeClassifier(max_depth=10)
      # clf = GaussianNB()
      # clf = LinearSVC(random_state=0, tol=1e-5)
      return clf


  def train(self,X,Y):
    # Define weak learner
    clf = DecisionTreeClassifier(max_depth=2)
    # clf = GaussianNB()
    #clf = LinearSVC(random_state=0, tol=1e-5)

    # Define the initial weights as 1/(Number of Datapoints)
    # In the beginning everypoint has same weight
    self.train_set_len = X.shape[0]
    self.D = np.ones(self.train_set_len) / self.train_set_len   

    for i in range(self.epoch_num):

      clf.fit(X,Y, sample_weight=self.D)  # Here we are providing sample weights to the decision tree 
      predicted = clf.predict(X) # Get the predicted vector
      print(predicted)
      print(f"Accuracy for Epoch {i}",self.accuracy(Y,predicted))
      r = np.sum(self.D *calculate_mult(Y,predicted)) # Y * predicted
      print(r)
      
      # calculate the new value of alpha
      # z = np.sqrt(1 - r**2)
      a = 0.5 * np.log((1 + r) / (1 - r)) 
      self.alpha[i] = a
      self.clfs.append(clf)
      
      # Reinitialize the weights 
      self.D = self.D * np.exp(-a *calculate_mult(Y,predicted))#* Y * predicted)
      sum = np.sum(self.D)
      #self.D = self.D / sum
      print(self.D)

model=AdaboostMH(args)
model.train(X,y)
    

[[0. 1. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 1. 1. 1.]
 [0. 1. 0. ... 1. 1. 0.]
 ...
 [0. 0. 1. ... 1. 1. 0.]
 [0. 1. 0. ... 1. 1. 0.]
 [0. 1. 0. ... 1. 1. 0.]]
Accuracy for Epoch 0 0.7630256410256299
-0.23697435897435892
[0.00063052 0.00063052 0.00059633 ... 0.00066667 0.00060752 0.00063052]
[[0. 1. 0. ... 1. 1. 0.]
 [1. 0. 0. ... 1. 1. 1.]
 [0. 1. 0. ... 1. 1. 0.]
 ...
 [0. 0. 1. ... 1. 1. 0.]
 [0. 1. 0. ... 1. 1. 0.]
 [0. 1. 0. ... 1. 1. 0.]]
Accuracy for Epoch 1 0.7630256410256305
-0.21970622322791045
[0.00059884 0.00058864 0.00053792 ... 0.00066667 0.00055751 0.00059884]
[[0. 1. 0. ... 1. 1. 0.]
 [1. 0. 0. ... 1. 1. 1.]
 [0. 1. 0. ... 1. 1. 0.]
 ...
 [0. 0. 1. ... 1. 1. 0.]
 [0. 1. 0. ... 1. 1. 0.]
 [0. 1. 0. ... 1. 1. 0.]]
Accuracy for Epoch 2 0.7630256410256305
-0.2048555138559817
[0.00057081 0.00055219 0.00048873 ... 0.00066667 0.00051469 0.00057081]
[[0. 1. 0. ... 1. 1. 0.]
 [1. 0. 0. ... 1. 1. 1.]
 [0. 1. 0. ... 1. 1. 0.]
 ...
 [0. 0. 1. ... 1. 1. 0.]
 [0. 1. 0. ... 1. 1. 0.]
 [0.